# Report ISJEAAI 

imports

In [56]:
import re
import json

In [57]:
REPORT_FILE_PATH = './report_final.json'

In [58]:
def reload():
    try:
        with open(REPORT_FILE_PATH) as json_file:
            data = json.load(json_file)
            data = {k: v for k, v in sorted(data.items(), key=lambda item: item[0])}
        return data
    except:
        print("An exception occurred when trying to open %s" % REPORT_FILE_PATH)
        return []

## Utils

In [59]:
def get_experiment_metadata(key):
    pattern = r"^(\w+)_input_(\w+)_classes_(\w+)_nn_(\w+)$"
    match = re.match(pattern, key)
    if match:
        return [match[1], match[2], match[3], match[4]]
    return [None, None, None, None]

In [69]:
def generate_table(target_dataset):
    data = reload()
    table = '<html> <table> <tr>'
    #table += '<th> dataset </th>'
    table += '<th> model </th>'
    table += '<th> pages </th>'
    table += '<th> output </th>'
    #table += '<th> val accuracy </th>'    
    #table += '<th> val kappa </th>'        
    table += '<th> test accuracy </th>'    
    table += '<th> test kappa </th>'            
    table += '<th> f1-score </th>'
    table += '<th> f1 ND </th>'
    table += '<th> f1 SD </th>'    
    table += '</tr>'

    max_acc = -1
    max_kappa = -1    
    max_t_kappa = -1
    max_t_acc = -1    
    max_f1_score = -1
    max_f1_score_FirstPage = -1
    max_f1_score_NextPage = -1

    max_t_kappa_exp = None
    max_t_acc_exp = None    
    max_acc_exp = None
    max_kappa_exp = None
    max_f1_score_exp = None
    max_f1_score_FirstPage_exp = None
    max_f1_score_NextPage_exp = None
    
    for key, value in data.items():
        
        dataset, pages, classes, model = get_experiment_metadata(key)       
        if dataset != target_dataset:
            continue
        f1_score = 100*value["weighted avg"]["f1-score"]
        f1_score_FirstPage = 100*value['ND']['f1-score']
        f1_score_NextPage = 100*value['SD']['f1-score']        
        acc = 100*value["valid_acc"]
        kappa = 100*value["valid_kappa"]
        tacc = 100*value["test_acc"]
        tkappa = 100*value["test_kappa"]        

        if f1_score_FirstPage > max_f1_score_FirstPage:
            max_f1_score_FirstPage = f1_score_FirstPage
            max_f1_score_FirstPage_exp = key
        if f1_score_NextPage > max_f1_score_NextPage:
            max_f1_score_NextPage = f1_score_NextPage
            max_f1_score_NextPage_exp = key
        if f1_score > max_f1_score:
            max_f1_score = f1_score
            max_f1_score_exp = key            
        if acc > max_acc:
            max_acc = acc
            max_acc_exp = key
        if kappa > max_kappa:
            max_kappa = kappa
            max_kappa_exp = key
        if tacc > max_t_acc:
            max_t_acc = tacc
            max_t_acc_exp = key
        if tkappa > max_t_kappa:
            max_t_kappa = tkappa
            max_t_kappa_exp = key

    for key, value in data.items():

        f1_score = 100*value["weighted avg"]["f1-score"]
        f1_score_FirstPage = 100*value['ND']['f1-score']
        f1_score_NextPage = 100*value['SD']['f1-score']        
        acc = 100*value["valid_acc"]
        kappa = 100*value["valid_kappa"]
        tacc = 100*value["test_acc"]
        tkappa = 100*value["test_kappa"]        

        diff_f1 = max_f1_score - f1_score 
        dfff_f1_FirstPage = max_f1_score_FirstPage  - f1_score_FirstPage
        diff_f1_NextPage = max_f1_score_NextPage - f1_score_NextPage
        diff_acc = max_acc - acc
        diff_kappa = max_kappa - kappa
        diff_tacc = max_t_acc - tacc
        diff_tkappa = max_t_kappa - tkappa
        
        diff_f1 = f'({diff_f1:.1f})' if diff_f1 > 0 else ''
        dfff_f1_FirstPage = f'({dfff_f1_FirstPage:.1f})' if dfff_f1_FirstPage > 0 else ''
        diff_f1_NextPage = f'({diff_f1_NextPage:.1f})' if diff_f1_NextPage > 0 else ''
        diff_acc = f'({diff_acc:.1f})' if diff_acc > 0 else ''
        diff_kappa = f'({diff_kappa:.1f})' if diff_kappa > 0 else ''
        diff_tacc = f'({diff_tacc:.1f})' if diff_tacc > 0 else ''
        diff_tkappa = f'({diff_tkappa:.1f})' if diff_tkappa > 0 else ''

        dataset, pages, classes, model = get_experiment_metadata(key)       
        if dataset != target_dataset:
            continue

        table += '<tr>'
        #table += f'<td> {dataset} </td>'
        table += f'<td> {model} </td>'
        table += f'<td> {pages} </td>'
        table += f'<td> {classes} </td>'
        #table += '<td>' + ('<strong>' if max_acc_exp == key else '' ) + f'{acc:.1f}{diff_acc}%'  + ('</strong>' if max_acc_exp == key else '') +'</td>'            
        #table += '<td>' + ('<strong>' if max_kappa_exp == key else '' ) + f'{kappa:.1f}{diff_kappa}%' + ('</strong>' if max_kappa_exp == key else '') +'</td>'        
        table += '<td>' + ('<strong>' if max_t_acc_exp == key else '' ) + f'{tacc:.1f}{diff_tacc}%'  + ('</strong>' if max_t_acc_exp == key else '') +'</td>'            
        table += '<td>' + ('<strong>' if max_t_kappa_exp == key else '' ) + f'{tkappa:.1f}{diff_tkappa}%' + ('</strong>' if max_t_kappa_exp == key else '') +'</td>'                
        table += '<td>' + ('<strong>' if max_f1_score_exp == key else '' ) + f'{f1_score:.1f}{diff_f1}%' + ('</strong>' if max_f1_score_exp == key else '') +'</td>'        
        table += '<td>' + ('<strong>' if max_f1_score_FirstPage_exp == key else '' ) + f'{f1_score_FirstPage:.1f}{dfff_f1_FirstPage}%' + ('</strong>' if max_f1_score_FirstPage_exp == key else '') +'</td>'        
        table += '<td>' + ('<strong>' if max_f1_score_NextPage_exp == key else '' ) + f'{f1_score_NextPage:.1f}{diff_f1_NextPage}%' + ('</strong>' if max_f1_score_NextPage_exp == key else '') +'</td>'                
        table += '</tr>'
    table+= '</table></html>'
    return table

In [61]:
print(generate_table('tobacco800'))

<html> <table> <tr><th> model </th><th> pages </th><th> output </th><th> val accuracy </th><th> val kappa </th></tr></table></html>


In [70]:
print(generate_table('ai_lab_splitter'))

<html> <table> <tr><th> model </th><th> pages </th><th> output </th><th> test accuracy </th><th> test kappa </th><th> f1-score </th><th> f1 ND </th><th> f1 SD </th></tr><tr><td> VGG16 </td><td> 1 </td><td> 2 </td><td>86.3(8.9)%</td><td>67.6(14.2)%</td><td>84.8(10.5)%</td><td>69.7(16.4)%</td><td>90.6(6.6)%</td></tr><tr><td> effnetB0 </td><td> 1 </td><td> 2 </td><td>88.4(6.8)%</td><td>67.5(14.3)%</td><td>89.0(6.3)%</td><td>70.5(15.6)%</td><td>92.8(4.4)%</td></tr><tr><td> VGG16 </td><td> 2 </td><td> 2 </td><td>92.4(2.8)%</td><td>80.1(1.6)%</td><td>92.7(2.6)%</td><td>79.8(6.3)%</td><td>95.4(1.9)%</td></tr><tr><td> effnetB0 </td><td> 2 </td><td> 2 </td><td>95.0(0.2)%</td><td>80.7(1.1)%</td><td>95.1(0.2)%</td><td>85.3(0.8)%</td><td>97.1(0.1)%</td></tr><tr><td> VGG16 </td><td> 3 </td><td> 2 </td><td>90.7(4.5)%</td><td>74.7(7.0)%</td><td>91.1(4.2)%</td><td>75.8(10.3)%</td><td>94.3(3.0)%</td></tr><tr><td> effnetB0 </td><td> 3 </td><td> 2 </td><td>94.1(1.1)%</td><td>79.4(2.4)%</td><td>94.3(1.1)%

<html> <table> <tr><th> model </th><th> pages </th><th> output </th><th> val accuracy </th><th> val kappa </th></tr><tr><td> VGG16 </td><td> 1 </td><td> 2 </td><td>85.7(3.9)%</td><td>66.2(10.3)%</td></tr><tr><td> effnetB0 </td><td> 1 </td><td> 2 </td><td>83.8(5.8)%</td><td>62.0(14.5)%</td></tr><tr><td> VGG16 </td><td> 2 </td><td> 2 </td><td>87.7(1.9)%</td><td>75.6(0.9)%</td></tr><tr><td> effnetB0 </td><td> 2 </td><td> 2 </td><td>88.7(0.9)%</td><td>74.1(2.3)%</td></tr><tr><td> VGG16 </td><td> 3 </td><td> 2 </td><td>89.1(0.6)%</td><td>73.3(3.2)%</td></tr><tr><td> effnetB0 </td><td> 3 </td><td> 2 </td><td>88.6(1.0)%</td><td>73.7(2.8)%</td></tr><tr><td> VGG16 </td><td> 3 </td><td> 4 </td><td><strong>89.6%</strong></td><td>75.0(1.5)%</td></tr><tr><td> effnetB0 </td><td> 3 </td><td> 4 </td><td>88.7(0.9)%</td><td><strong>76.5%</strong></td></tr></table></html>

<html> <table> <tr><th> model </th><th> pages </th><th> output </th><th> test accuracy </th><th> test kappa </th><th> f1-score </th><th> f1 ND </th><th> f1 SD </th></tr><tr><td> VGG16 </td><td> 1 </td><td> 2 </td><td>86.3(8.9)%</td><td>67.6(14.2)%</td><td>84.8(10.5)%</td><td>69.7(16.4)%</td><td>90.6(6.6)%</td></tr><tr><td> effnetB0 </td><td> 1 </td><td> 2 </td><td>88.4(6.8)%</td><td>67.5(14.3)%</td><td>89.0(6.3)%</td><td>70.5(15.6)%</td><td>92.8(4.4)%</td></tr><tr><td> VGG16 </td><td> 2 </td><td> 2 </td><td>92.4(2.8)%</td><td>80.1(1.6)%</td><td>92.7(2.6)%</td><td>79.8(6.3)%</td><td>95.4(1.9)%</td></tr><tr><td> effnetB0 </td><td> 2 </td><td> 2 </td><td>95.0(0.2)%</td><td>80.7(1.1)%</td><td>95.1(0.2)%</td><td>85.3(0.8)%</td><td>97.1(0.1)%</td></tr><tr><td> VGG16 </td><td> 3 </td><td> 2 </td><td>90.7(4.5)%</td><td>74.7(7.0)%</td><td>91.1(4.2)%</td><td>75.8(10.3)%</td><td>94.3(3.0)%</td></tr><tr><td> effnetB0 </td><td> 3 </td><td> 2 </td><td>94.1(1.1)%</td><td>79.4(2.4)%</td><td>94.3(1.1)%</td><td>82.5(3.6)%</td><td>96.7(0.6)%</td></tr><tr><td> VGG16 </td><td> 3 </td><td> 4 </td><td>91.1(4.1)%</td><td>77.9(3.9)%</td><td>91.5(3.8)%</td><td>77.2(8.9)%</td><td>94.5(2.8)%</td></tr><tr><td> effnetB0 </td><td> 3 </td><td> 4 </td><td><strong>95.2%</strong></td><td><strong>81.8%</strong></td><td><strong>95.3%</strong></td><td><strong>86.1%</strong></td><td><strong>97.2%</strong></td></tr></table></html>